In [37]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import pickle

In [38]:
## load the ann train model,scalar pickle and onehot
model = load_model('model.h5')

## load the encoder and scalar
with open('one_hot_encode_geo.pkl','rb') as file:
    one_hot_encode_geo = pickle.load(file)
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)
with open('scalar.pkl','rb') as file:
    scalar = pickle.load(file)

In [39]:
## Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts':2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 5000
}

In [40]:
geo_encoded = one_hot_encode_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns = one_hot_encode_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\Dell\Desktop\data\science\ds\ANN classification\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [41]:
input_df = pd.DataFrame([input_data])

In [42]:
## encode categorical variable
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,5000


In [43]:
## concatination one hot encoded
input_df = pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [44]:
input_df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,5000,1.0,0.0,0.0


In [45]:
## Scaling the input data 
input_scaled = scalar.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -1.65923237,  1.00150113,
        -0.57946723, -0.57638802]])

In [46]:
## Predict churn
predict = model.predict(input_scaled)
predict

1/1 [==============================] - 1s 817ms/step


array([[0.02187472]], dtype=float32)

In [47]:
pridict_proba = predict[0][0]

In [48]:
if pridict_proba>0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
